In [1]:
# En caso de problemas, utilizar las dependencias de librerías de este requierement: https://github.com/googlecolab/backend-info/blob/d6d345cb94fc5fd49951c9af0f6ead5e962bfab2/pip-freeze.txt
!pip install numpy==1.23.5
!pip install transformers[torch]==4.35.2
!pip install accelerate -U
!pip install evaluate

  Using cached transformers-4.35.2-py3-none-any.whl (7.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 13.1 MB/s eta 0:00:00
  Using cached accelerate-0.30.1-py3-none-any.whl (302 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvi

In [2]:
import pandas as pd
def load_prepare_data(path):
  """
  Función para cargar y procesar datos para el ejercicio.
  """
  df = pd.read_csv(path,sep=",")
  map_classes = {
    "religion":1,
    "age":1,
    "ethnicity":1,
    "gender":1,
    "other_cyberbullying":1,
    "not_cyberbullying":0,
  }
  df["cyberbullying"] = df.cyberbullying_type.map(map_classes)
  return df[["tweet_text","cyberbullying"]].copy()

# Ejercicio


En este ejercicio vamos a trabajar con un conjunto de datos procedente de medios sociales online.

Uno de los mayores problemas en el internet de hoy en día es la presencia de actitudes negativas hacia algunos colectivos en relación a su etnia, género, religión o ideología política. En este ejercicio trabajaremos con un conjunto de datos reales, etiquetados manualmente, procedentes de la plataforma [Kaggle](https://www.kaggle.com/datasets/andrewmvd/cyberbullying-classification/data). Originalmente, a cada documento del dataset se le asignó una de las siguientes categorías:
- *religion*
- *age*
- *ethnicity*
- *gender*
- *other_cyberbullying*
- *not_cyberbullying*


El objetivo inicial del dataset era su uso para entrenar un modelo capaz de detectar el tipo de contenido de odio presente en internet según el colectivo al que se atacaba. En este caso, para simplificar el ejercicio, se ha generado una función `load_prepare_data()` que cambia las categorías del dataset obteníendose al final 2 categorías con valor 1 o 0, indicando si el tweet tiene contenido de odio

**En este ejercicio debeis entrenar un modelo de clasificación utilizando la librería Transformers.** Dado que el análisis exploratorio ha sido realizado en el ejercicio anterior, en este caso podréis centraros en entrenar el modelo utilizando la librería Transformers, seleccionando un modelo pre-entrenado adecuado, entrenando el modelo y llevando a cabo la evaluación.


**Nota 1**: Este ejercicio requiere el uso de las GPUs de Google Colab. Este Colab debería estar preconfigurado para ejecutarse en GPU, pero si tuviera problemas en la ejecución que me contacte a través del Moodle para buscar soluciones alternativas.

## 0. Imports


In [3]:
from transformers import (
   AutoConfig,
   AutoTokenizer,
   AutoModelForSequenceClassification,
   AdamW
)
import torch
import pandas as pd
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


## 1. Obtención del corpus
Para la obtención de los datos teneis disponible la función `load_prepare_data()`. Esta función prepara los datos del ejercicio en formato Pandas dataframe para que podais realizarlo.

In [4]:
path_data = "https://raw.githubusercontent.com/luisgasco/ntic_master_datos/main/datasets/cyberbullying_tweets.csv"
# Path de datos alternativos en caso de que el anterior no funcione (al estar alojado en github puede haber limitaciones
# en la descarga.
# path_data = "https://zenodo.org/records/10938455/files/cyberbullying_tweets.csv?download=1"
dataset = load_prepare_data(path_data)

In [5]:
dataset.head(4)

,tweet_text,cyberbullying
0,"In other words #katandandre, your food was cra...",0
1,Why is #aussietv so white? #MKR #theblock #ImA...,0
2,@XochitlSuckkks a classy whore? Or more red ve...,0
3,"@Jason_Gio meh. :P thanks for the heads up, b...",0


## 2. Análisis exploratorio

Podéis saltarlo en este ejercicio.

## 3. Preprocesado y Normalización

Deberemos dividir en primer lugar nuestro dataset para el proceso. En primer lugar vamos a tomar los campos textuales en nuestro corpus y por otro lado vamos a coger las etiquetas de nuestro corpus. 

In [6]:
texts = dataset.tweet_text.values  # an array of strings  df.tweet_text.values
labels = dataset.cyberbullying.values  # an array of integers  df.molestia.values


Después de dividir el conjunto de datos en Train-Validation (90%) y Test (10%), procederemos a dividir nuestro corpus en conjuntos de entrenamiento y prueba. El conjunto de prueba no será utilizado hasta la fase de validación. 

In [7]:
train_texts, test_texts, train_labels, test_labels = train_test_split(texts,
                                                                      labels,
                                                                      test_size=.25,
                                                                      random_state=0,
                                                                      stratify = labels)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts,
                                                                    train_labels,
                                                                    test_size=.2,
                                                                    random_state=0,
                                                                    stratify = train_labels)

En este caso, estaremos utilizando un modelo de deep learning de la librería transformers. Hemos seleccionado el modelo cardiffnlp/twitter-roberta-base-sentiment-latest. Este modelo ha sido pre-entrenado y vectorizado previamente, y está específicamente diseñado para llevar a cabo análisis de sentimiento en datos de texto.

In [8]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"


Una vez que hayamos seleccionado el modelo, podremos cargar nuestro tokenizador utilizando la clase AutoTokenizer. De esta manera, podremos cargar el tokenizador específico de cardiffnlp.

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Ahora, vamos a comprobar como el tokenizador que hemos seleccionado subdivide un texto de ejemplo:

In [10]:
texto = "Cats and zebras love to nap in sunny spots"
texto_tokens = tokenizer(texto).tokens()
texto_tokens

['<s>',
 'C',
 'ats',
 'Ġand',
 'Ġze',
 'br',
 'as',
 'Ġlove',
 'Ġto',
 'Ġnap',
 'Ġin',
 'Ġsunny',
 'Ġspots',
 '</s>']

Aquí podemos observar cómo el tokenizador divide las palabras. Se nota que divide las palabras completas y agrega la letra "Ġ" al inicio de cada palabra para indicar el comienzo de una nueva palabra. Además, se añaden dos tokens al inicio y al final de la frase. Esto indica que, en lugar de tokenizar por palabras, la biblioteca transformers divide los tokens en sub-palabras. Esta es una técnica más eficiente para representar todas las combinaciones de términos en un idioma.

Ahora que ya tenemos el tokenizador y los textos, vamos a trabajar con el ecosistema Hugging Face, que utiliza un formato de datos basado en la biblioteca dataset. A continuación, presento una función que heredará de la clase "Dataset" de Hugging Face y preprocesará nuestros textos.

Cuando se cree esta clase, se introducirán los textos, las etiquetas, el tokenizador y el tamaño máximo esperado de los textos. Internamente, la función llevará a cabo un proceso de tokenización o codificación de nuestro texto para que pueda ser comprendido por el modelo, y lo devolverá en un formato Hugging Face. En este formato, Hugging Face verá una sustitución de esos tokens por identificadores numéricos, así como la indicación de a qué partes de esos datos de entrada debe prestar atención, y las etiquetas que se utilizarán para el entrenamiento.

In [11]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        """
        Constructor de la clase CustomDataset.
        Parámetros:
        - texts: Lista de textos.
        - labels: Lista de etiquetas correspondientes a los textos.
        - tokenizer: Objeto del tokenizador a utilizar.
        - max_length: Longitud máxima de la secuencia después de la tokenización.
        """
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        """
        Devuelve la longitud del conjunto de datos.
        """
        return len(self.texts)

    def __getitem__(self, idx):
        """
        Obtiene un elemento del conjunto de datos.

        Parámetros:
        - idx: Índice del elemento a obtener.

        Devuelve:
        Un diccionario con 'input_ids', 'attention_mask' y 'labels'.
        """
        # Obtener el texto y la etiqueta del índice proporcionado
        text = str(self.texts[idx])
        label = int(self.labels[idx])

        # Tokenizar el texto
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )

        # Devolver el diccionario con los datos
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

Una vez definida la función, ahora indicaremos que deseamos un tamaño máximo de 128 tokens, lo cual será suficiente para trabajar con tweets. A continuación, prepararemos el conjunto de datos de entrenamiento de la forma que se ha comentado anteriormente.

In [12]:
max_length = 128  # Puedes ajustar esto según tus necesidades

train_dataset = CustomDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer, max_length)
test_dataset = CustomDataset(test_texts, test_labels, tokenizer, max_length)

Podremos ver que cada elemento del nuevo formato tendrá:
- *inputs_ids*: Identificadores numéricos de los tokens en el vocabulario del modelo
- *attention_mask*: Vector que indica a la red neuronal qué partes de la secuencia de entrada debe prestar atención y cuáles ignorar.
- *labels*: Este campo contiene la etiqueta asociada al texto


Para ilustrar un ejemplo de salida, vamos a tomar el elemento número 20. Observamos que tendrá unos input_ids, donde los números representarán las palabras que hemos visto previamente. El token 0 corresponderá al token de inicio y el token 2 será el token de separación. Las attention_mask tendrán valores unitarios en los elementos donde hay un identificador de término asociado.

In [13]:
train_dataset[20]

{'input_ids': tensor([    0,  1213,   214,   608,     5,  4187,   334, 23934,  3650,     4,
         22008,    70,    51,    64,     7,   120,    82,    11,  3605,     8,
           120,   409,    19,    24,    30,  7547, 10802,    23,     5,  7850,
             8,   667,     7,   310,     5,  4887,   177,     4,     2,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

Antes de preprocesar la entrada, necesitamos definir la longitud de la secuencia (longitud maxima del documento en tokens) y el Batch size. En Colab, hay limitaciones (batch = 8 y secuencia=96). Si se reduciera el batch se podría incrementar el tamaño, pero en este caso nos es indiferente.

In [14]:
max_seq_length = 96
train_batch_size =  8
eval_batch_size = 8
test_batch_size = 8

## 4. Entrenamiento


Para iniciar el entrenamiento, ya que todos los datos están preparados y el conjunto de entrenamiento ha sido dividido, vamos a invocar la clase AutoModelForSequenceClassification(). Esta clase permite cargar un modelo que puede ser fácilmente ajustado para clasificar secuencias de texto.

En esta clase, vamos a especificar el número de categorías que esperamos extraer, que en este caso sabemos que son 2. Además, hemos generado dos diccionarios para que durante el proceso de evaluación podamos acceder a la etiqueta en lugar del número, permitiéndonos comprender realmente qué significa cada etiqueta.

In [15]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

id2label = {0: "NO_cyberbulling", 1: "cyberbulling"}
label2id = {"NO_cyberbulling": 0, "cyberbulling": 1}
model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           num_labels=2,
                                                           id2label=id2label,
                                                           label2id=label2id,
                                                           ignore_mismatched_sizes=True)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpo

Posteriormente, definiremos algunos parámetros necesarios para llevar a cabo el entrenamiento:

In [16]:
import accelerate

training_args = TrainingArguments(
    output_dir="modelo_test",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False
)


Para la evaluación, al final de cada época utilizando el conjunto de validación que hemos generado, también necesitamos definir una función de evaluación. En este caso, extraeremos dos métricas de la librería evaluate.

Esta función tomará una predicción junto con unas etiquetas. Identificaré la etiqueta más probable y calcularé la precisión (accuracy) y la puntuación F (F-score).

In [17]:
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")
f1_score = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy_value = accuracy.compute(predictions=predictions, references=labels)
    f1_score_value = f1_score.compute(predictions=predictions, references=labels)

    return {
        "accuracy": accuracy_value,
        "f1_score": f1_score_value,
    }

Una vez se tenga ya todo esto, vamos a poder crear un objeto trainer que va a ser el encargado de llevar a cabo este proceso de entrenamiento. Al objeto trainer le voy a introducir el modelo, le voy a introducir los argumentos de entrenamiento, el conjunto de datos de entrenamiento, el de evaluación, el tokenizador y la función de evaluación llamada compute_metrics

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


Una vez que tengo mi objeto "trainer" creado,  se entrenará con una sola época para obtener los resultados de forma más rápida..

In [19]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,0.350400,0.363982,{'accuracy': 0.8976796197931227},{'f1': 0.940632603406326}


Trainer is attempting to log a value of "{'accuracy': 0.8976796197931227}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.940632603406326}" of type <class 'dict'> for key "eval/f1_score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=7154, training_loss=0.3768614661536938, metrics={'train_runtime': 981.4685, 'train_samples_per_second': 29.155, 'train_steps_per_second': 7.289, 'total_flos': 1882230712281600.0, 'train_loss': 0.3768614661536938, 'epoch': 1.0})

Vemos que con una epoca tenemos un accuracy de 0.89, junto con un F1 Score de 0.94. Estos resultados son sobre el validation set, nosotros vamos a querer evaluar el modelo sobre nuestros datos de evaluación 

## 5. Entrenamiento y evaluación de modelos



Una vez entrenado el modelo, procederemos a llevar a cabo su evaluación. Utilizaremos nuestro corpus de prueba y ejecutaremos nuestro proceso de evaluación. Aquí podemos observar que obtenemos un F-Score de 0.93 y una precisión de 0.89


In [20]:
# Make predictions on the test data
trainer.evaluate(test_dataset) # evaluate con test_dataset

Trainer is attempting to log a value of "{'accuracy': 0.8923928541474461}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9377939393939394}" of type <class 'dict'> for key "eval/f1_score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.38073909282684326,
 'eval_accuracy': {'accuracy': 0.8923928541474461},
 'eval_f1_score': {'f1': 0.9377939393939394},
 'eval_runtime': 108.9908,
 'eval_samples_per_second': 109.395,
 'eval_steps_per_second': 27.351,
 'epoch': 1.0}


Si deseamos obtener métricas como un classification report, podríamos utilizar nuestro "trainer" para hacer predicciones sobre nuestro conjunto de datos y guardarlas en las predicciones, como se muestra en las siguientes líneas de código.

In [21]:
# Make predictions on the test data
predictions = trainer.predict(test_dataset)

El siguiente código lo que nos va a devolver es la probabilidad de dar más en la red neuronal de salida de cada una de las categorias y lo que tendremos que seleccionar es el valor máximo de estas predicciones. 

In [22]:
predictions[0][0]

array([-3.7813811,  3.9937506], dtype=float32)

Por lo que a continuación tomamos axis = 1 porque es el valor mayor. Y lo guardamos en una lista

In [23]:
# Access the predicted labels
y_pred = predictions.predictions.argmax(axis=1)

Luego extraemos las etiquetas reales de la lista y calculamos ese classification report que teniamos previamente

In [24]:
y_true = [x["labels"].item() for x in test_dataset]

In [25]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))

[[ 969 1017]
 [ 266 9671]]
              precision    recall  f1-score   support

           0       0.78      0.49      0.60      1986
           1       0.90      0.97      0.94      9937

    accuracy                           0.89     11923
   macro avg       0.84      0.73      0.77     11923
weighted avg       0.88      0.89      0.88     11923



Podemos comparar los resultados con el clasificador obtenido mediante ingeniería de características:

precision    recall  f1-score   support

         0.0       0.52      0.69      0.59      1984
         1.0       0.93      0.87      0.90      9474

    accuracy                           0.83     11458
    macro avg       0.72      0.78      0.74     11458
    weighted avg       0.86      0.83      0.84     11458

Al comparar los dos resultados, observamos que con el nuevo modelo entrenado obtenemos un mejor accuracy. Además, conseguimos una mejor precisión en la clase 0, que es minoritaria en comparación con el clasificador obtenido mediante ingeniería de características. Esto sugiere que el modelo Transformer es capaz de generar mejores resultados. Sin embargo, al ser un modelo de caja negra, nos enfrentamos a una situación donde sacrificamos la explicabilidad rendimiento.

Es importante mencionar que este modelo se entrenó solo con una época. Si hubiéramos entrenado con más épocs, seguramente habríamos obtenido resultados aún mayores tanto en el accuracy como en el recall y el F1-score.